<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/main/M_Net_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rs = 23  # fester Random Seed

Load Data

In [6]:
from tensorflow.keras.datasets import fashion_mnist, mnist

# Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [7]:
def ifelse(a):
  if a == 0:
    return 1
  else:
    return a

In [14]:
def rms_norm(v_x):
  n_rms = np.sqrt(np.sum(v_x ** 2)/(len(v_x) - 1))
  return v_x/n_rms

Preprocess data

NEW Norm

In [8]:
X = X_train.reshape(len(X_train), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm)
X = np.array(X)

In [25]:
X

array([[0.99999167, 0.99999167, 0.99999167, ..., 0.99999167, 0.99999167,
        0.99999167],
       [0.99999167, 0.99999167, 0.99999167, ..., 0.99999167, 0.99999167,
        0.99999167],
       [0.99999167, 0.99999167, 0.99999167, ..., 0.99999167, 0.99999167,
        0.99999167],
       ...,
       [0.99999167, 0.99999167, 0.99999167, ..., 0.99999167, 0.99999167,
        0.99999167],
       [0.99999167, 0.99999167, 0.99999167, ..., 0.99999167, 0.99999167,
        0.99999167],
       [0.99999167, 0.99999167, 0.99999167, ..., 0.99999167, 0.99999167,
        0.99999167]])

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X = X_train.reshape(len(X_train), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X = preproccessing.fit_transform(X)
# X = np.array(X)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

Originaldaten:
Shape: (60000, 28, 28), Mean: 33.318421, STD: 78.567490
Vorbereitete Daten:
Shape: (60000, 784), Mean: 0.496145, STD: 0.868230


Create Hidden Activations

In [19]:
H = np.random.rand(60000, 47)
print(H.shape)
print(H)

(60000, 47)
[[0.8990862  0.52916777 0.91271004 ... 0.34291353 0.27399914 0.43212453]
 [0.48197691 0.21871386 0.3013048  ... 0.2458945  0.50041352 0.84457424]
 [0.54716046 0.93362684 0.12118643 ... 0.36353454 0.49862239 0.20064858]
 ...
 [0.185872   0.92310609 0.91811914 ... 0.83258575 0.5239248  0.25513659]
 [0.41492943 0.15071553 0.17736268 ... 0.18779191 0.4993161  0.47676953]
 [0.30429026 0.86461749 0.95748426 ... 0.44737201 0.40195264 0.20417338]]


1st Forward Learn

In [20]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(H, X)
print(reg.score(H, X))
X_pred = reg.predict(H)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.0007727881839603041
MAE 0.36551675579705567
MSE 0.6767767051258416
R2 -1227.0673734088828
R2 vw -1292.0156445916157
R2 ua -1227.0673734088828
(784, 47)
(784,)


1st Back Activations

In [21]:
XT = X.transpose()
print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

shape of X_train (60000, 784) and X_train transposed (784, 60000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.32792876349178435
MAE 0.44324360095599574
MSE 0.49689092566819454
R2 -1.0025576161822731
R2 vw -1.049442779437837
R2 ua -1.0025576161822731
(60000, 47)
(60000,)


2nd Forward learn - result are Coherent Weights

In [22]:
reg = LinearRegression().fit(H_new, X)
print(reg.score(H_new, X))
X_pred = reg.predict(H_new)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.4721974369453756
MAE 0.29807716380528154
MSE 0.35748073647022055
R2 -33.48179097338724
R2 vw -0.11775821247348628
R2 ua -33.48179097338724
(784, 47)
(784,)


2nd Back Activation - final representation of X_train in Hidden Activations

In [23]:
reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.4985042441436013
MAE 0.36411764636943467
MSE 0.3707771986208351
R2 vw -0.006000975413833734
R2 ua 0.014332753153585868
(60000, 47)
(60000,)


KNN test with Hidden Activations

In [26]:
X = X_test.reshape(len(X_test), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm)
X = np.array(X)

In [27]:
# X = [[0], [1], [2], [3]]
# y = [0, 0, 1, 1]

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X_p_test = X_test.reshape(len(X_test), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X_p_test = preproccessing.fit_transform(X_p_test)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
neigh.fit(H_new, y_train)

# print(neigh.predict([[1.1]]))

# print(neigh.predict_proba([[0.9]]))

Originaldaten:
Shape: (10000, 28, 28), Mean: 33.791224, STD: 79.172463
Vorbereitete Daten:
Shape: (10000, 784), Mean: 0.511132, STD: 0.859444


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='distance')

Back Activation of Test Data

In [28]:
XT = X.transpose()
print(f'shape of X_p_train {X.shape} and X_p_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_test = reg.coef_
print(H_test.shape)
HI_test = reg.intercept_
print(HI_test.shape)

shape of X_p_train (10000, 784) and X_p_train transposed (784, 10000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.5070284170300943
MAE 0.3701270080801701
MSE 0.3570749729748137
R2 0.028997383553173012
R2 vw 0.0277239570565419
R2 ua 0.028997383553173012
(10000, 47)
(10000,)


In [29]:
# Predicting the Test set results
y_pred = neigh.predict(H_test)

In [ ]:
y_prob = neigh.predict_proba(H_test)

In [ ]:
print(y_prob[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [30]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 954    0    4    1    2    6    9    3    1    0]
 [   0 1121    5    0    1    1    5    0    2    0]
 [  10    5  962   15    2    0    8   11   17    2]
 [   0    2   10  946    1   12    2    9   21    7]
 [   2    1    6    1  911    3    7    6    3   42]
 [   7    0    5   31    8  806   14    2    9   10]
 [  13    4    4    1    6    6  920    0    4    0]
 [   0   11   15    6    5    1    0  946    4   40]
 [  10    0   11   22   12   28    4    9  870    8]
 [   5    5    4   10   21    7    0   34    4  919]]


In [31]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9355
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       980
           1       0.98      0.99      0.98      1135
           2       0.94      0.93      0.93      1032
           3       0.92      0.94      0.93      1010
           4       0.94      0.93      0.93       982
           5       0.93      0.90      0.91       892
           6       0.95      0.96      0.95       958
           7       0.93      0.92      0.92      1028
           8       0.93      0.89      0.91       974
           9       0.89      0.91      0.90      1009

    accuracy                           0.94     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.94      0.94      0.94     10000

